<a href="https://colab.research.google.com/github/nazanaza2970/BENTO-PACKAGING-ACTIVITY-RECOGNITION-CHALLENGE/blob/naza/Bento_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/windowed_500_xz",compression = "xz")
q = pd.read_feather("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/quaternions.feather")

In [ ]:
cols = list(q.columns )
q2 = []
for i in tqdm(cols):
    col = np.stack(q[i])
    q2.append(col[:,0])
    q2.append(col[:,1])
    q2.append(col[:,2])
    q2.append(col[:,3])

q2 = np.transpose(np.array(q2))
q2 = np.append(q2,q2[0]).reshape(1786500,60)

100%|██████████| 15/15 [00:39<00:00,  2.65s/it]


In [ ]:
q2.shape

(1786500, 60)

In [ ]:
seg = []
labels = []
for i in tqdm(range(0,len(data)-500,500)):
    seg.append(data.iloc[i:i+500,:-2])
    labels.append(data.iloc[i][-1])

100%|██████████| 3572/3572 [00:02<00:00, 1488.76it/s]


In [ ]:
seg[0].shape

(500, 39)

In [ ]:
len(labels)

3572

In [ ]:
seg = np.array(seg)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(seg, labels, test_size=.20, random_state=12)

In [ ]:
x_train = x_train/x_train.max(axis = 0)
x_val = x_val/x_val.max(axis = 0)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(np.array(y_train).reshape(-1,1))
y_val = onehot_encoder.fit_transform(np.array(y_val).reshape(-1,1))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv1D(30,3,3,input_shape = (500,39)))
model.add(keras.layers.LeakyReLU(alpha=0.3))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1D(60,3,3))
model.add(keras.layers.LeakyReLU(alpha=0.3))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv1D(120,1,3))
model.add(keras.layers.LeakyReLU(alpha=0.3))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Conv1D(60,3))
# model.add(keras.layers.Conv1D(100,4,4))
# model.add(keras.layers.Conv1D(100,4,4))
# model.add(keras.layers.Conv1D(200,4,4))
# model.add(keras.layers.Conv1D(200,4,4))
# model.add(keras.layers.Conv1D(200,4,4))
# model.add(keras.layers.Conv1D(200,4,4))
# model.add(keras.layers.Conv1D(200,4,4))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.LSTM(200,return_sequences = True))
model.add(keras.layers.LSTM(100,return_sequences = True))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.LSTM(100,return_sequences = True))
# model.add(keras.layers.LSTM(200,input_shape = (500,60)))
model.add(keras.layers.LSTM(100))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
# model.add(SeqSelfAttention(attention_activation='sigmoid'))
# model.add(keras.layers.Dropout(0.5))
# # model.add(Dense(200, activation='relu'))
# model.add(keras.layers.Dense(500, activation='relu'))
# model.add(keras.layers.Dense(500, activation='relu'))
# model.add(keras.layers.Dense(500, activation='relu'))
model.add(keras.layers.Dense(200))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(100))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs = 50,verbose = 2,validation_data = (x_val,y_val))

Epoch 1/50
90/90 - 7s - loss: 2.0035 - accuracy: 0.2296 - val_loss: 1.9274 - val_accuracy: 0.2755
Epoch 2/50
90/90 - 2s - loss: 1.6842 - accuracy: 0.3465 - val_loss: 1.7617 - val_accuracy: 0.3958
Epoch 3/50
90/90 - 2s - loss: 1.6059 - accuracy: 0.3857 - val_loss: 1.7527 - val_accuracy: 0.3133
Epoch 4/50
90/90 - 2s - loss: 1.5434 - accuracy: 0.4186 - val_loss: 1.5808 - val_accuracy: 0.3888
Epoch 5/50
90/90 - 2s - loss: 1.4684 - accuracy: 0.4393 - val_loss: 1.4570 - val_accuracy: 0.4238
Epoch 6/50
90/90 - 2s - loss: 1.4326 - accuracy: 0.4669 - val_loss: 2.1818 - val_accuracy: 0.3035
Epoch 7/50
90/90 - 2s - loss: 1.3999 - accuracy: 0.4764 - val_loss: 1.3872 - val_accuracy: 0.4490
Epoch 8/50
90/90 - 2s - loss: 1.3548 - accuracy: 0.5093 - val_loss: 1.5082 - val_accuracy: 0.4643
Epoch 9/50
90/90 - 2s - loss: 1.3150 - accuracy: 0.5096 - val_loss: 1.4871 - val_accuracy: 0.4601
Epoch 10/50
90/90 - 2s - loss: 1.2966 - accuracy: 0.5282 - val_loss: 1.3569 - val_accuracy: 0.4643
Epoch 11/50
90/90 -

In [ ]:
#Best model with quaternion
#*******************************************


# model = keras.models.Sequential()
# model.add(keras.layers.Conv1D(15,4,4,input_shape = (500,60)))
# model.add(keras.layers.Conv1D(15,4,4))
# model.add(keras.layers.LSTM(100,return_sequences = True))
# # model.add(keras.layers.LSTM(200,input_shape = (500,60)))
# model.add(keras.layers.LSTM(100))
# model.add(keras.layers.Dense(500, activation='relu'))
# model.add(keras.layers.Dense(500, activation='relu'))
# # model.add(keras.layers.Dense(50, activation='relu'))
# model.add(keras.layers.Dense(10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])